In [1]:
import os
import time
import pandas as pd


In [17]:
#in
main = os.getcwd()
TopicWithLabel = main + "/../../result/modelOutput/TopicWithLabel.xlsx"
TopicWithValue = main + "/../../result/modelOutput/TopicWithValue.csv"

#out
output_reuslt = main + "/../../result/integrationOutput"

In [20]:
import pandas as pd

# Load the Excel file
df = pd.read_excel(TopicWithLabel)

# List of industries
industries = ['AGRO', 'CONSUMP', 'FINCIAL', 'INDUS', 'PROPCON', 'RESOURC', 'SERVICE', 'TECH']

# Dictionary to hold topics for each industry
industry_topics = {industry: [] for industry in industries}

# Loop through each industry and extract topics
for industry in industries:
    industry_rows = df[df['Industry'].str.contains(industry, na=False)]
    topics = industry_rows['Topic'].tolist()
    industry_topics[industry] = topics


{'AGRO': ['Topic 17', 'Topic 26', 'Topic 36', 'Topic 42', 'Topic 51', 'Topic 59', 'Topic 96', 'Topic 104', 'Topic 107', 'Topic 112', 'Topic 115', 'Topic 123', 'Topic 127', 'Topic 131', 'Topic 135', 'Topic 136', 'Topic 137', 'Topic 156', 'Topic 180'], 'CONSUMP': [], 'FINCIAL': ['Topic 4', 'Topic 8', 'Topic 10', 'Topic 12', 'Topic 16', 'Topic 19', 'Topic 32', 'Topic 34', 'Topic 35', 'Topic 37', 'Topic 38', 'Topic 39', 'Topic 41', 'Topic 43', 'Topic 44', 'Topic 65', 'Topic 66', 'Topic 70', 'Topic 71', 'Topic 73', 'Topic 81', 'Topic 82', 'Topic 83', 'Topic 85', 'Topic 86', 'Topic 90', 'Topic 91', 'Topic 92', 'Topic 97', 'Topic 106', 'Topic 108', 'Topic 110', 'Topic 111', 'Topic 119', 'Topic 126', 'Topic 129', 'Topic 143', 'Topic 144', 'Topic 145', 'Topic 148', 'Topic 149', 'Topic 163', 'Topic 166', 'Topic 168', 'Topic 169', 'Topic 170', 'Topic 172', 'Topic 174', 'Topic 176', 'Topic 186', 'Topic 193', 'Topic 195'], 'INDUS': ['Topic 0', 'Topic 7', 'Topic 47', 'Topic 48', 'Topic 72', 'Topic 7

In [22]:
import pandas as pd

# Load your Excel file
df = pd.read_csv(TopicWithValue, nrows=5)

topic_columns = [col for col in df.columns if col.startswith('Topic_')]

for col in topic_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Iterate over the DataFrame rows
for index, row in df.iterrows():
    # Get the topic columns for the current row, drop missing values, and find the indices of the top 3 values
    top_values = row[topic_columns].dropna().nlargest(10)
    top_indices = top_values.index
    # Set all other values to 0
    for col in topic_columns:
        if col not in top_indices:
            df.at[index, col] = 0
            
df.to_excel('C:/Users/uSeR/Documents/VScode/sp2023-stock/TopicModeling/result/TopValueOutput/TopicWithTop10Value.xlsx', index=False)

In [23]:
df

,Article_ID,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,...,Topic_190,Topic_191,Topic_192,Topic_193,Topic_194,Topic_195,Topic_196,Topic_197,Topic_198,Topic_199
0,12277,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.00500,0.005000,0.00500,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12285,0.001250,0.001250,0.001250,0.001250,0.001250,0.001250,0.00125,0.501268,0.00125,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12287,0.000250,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12289,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000,0.00100,0.001000,0.00000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12291,0.000556,0.000556,0.000556,0.000556,0.000556,0.000556,0.00000,0.000000,0.00000,...,0.261518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import pandas as pd

# Load the Excel file
df = pd.read_excel('C:/Users/uSeR/Documents/VScode/sp2023-stock/TopicModeling/result/TopValueOutput/TopicWithTop10Value.xlsx')


# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Filter the DataFrame for the date range
start_date = '2018-01-01'
end_date = '2022-12-31'
df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

# Iterate over each industry
for industry, topics in industry_topics.items():
    # Convert topic numbers in the list to column names
    topic_columns = [f'Topic_{t.split()[1]}' for t in topics]

    # Select only the columns for the specified topics and the 'Date' column
    selected_columns = ['Date'] + topic_columns
    industry_df = df[selected_columns]

    # Group by 'Date' and sum the values
    industry_grouped = industry_df.groupby('Date').sum().reset_index()

    # Create a complete date range
    all_dates = pd.date_range(start=start_date, end=end_date)
    
    # Reindex to include all dates, filling missing ones with 0
    industry_grouped.set_index('Date', inplace=True)
    industry_grouped = industry_grouped.reindex(all_dates, fill_value=0).reset_index()

    # Rename 'index' column back to 'Date'
    industry_grouped.rename(columns={'index': 'Date'}, inplace=True)

    # Save to CSV
    industry_grouped.to_csv(f'C:/Users/uSeR/Documents/VScode/sp2023-stock/TopicModeling/result/test/{industry}.csv', index=False)
